# observing-hares
notebook for simulating observation(s) of hares drawn using `drawing-hares.ipynb`

In [51]:
#stock imports
import numpy as np
import pandas as pd
import json
import scipy
import os
##plotting
import matplotlib.pyplot as plt

#plt.style.use('dark_background')
plt.style.use('default')

## single hare
let's try to get this working for a single hare first, with fairly uninformed observational uncertainties

In [3]:
hare_idx = 42
hare_df = pd.read_json(f'form/hare{hare_idx}/hare{hare_idx}.json')
hare_df

,initial_mass,initial_Zinit,initial_Yinit,initial_MLT,star_age,radius,luminosity,star_feh,nu_0_4,nu_0_5,...,nu_0_32,nu_0_33,nu_0_34,nu_0_35,nu_0_36,nu_0_37,nu_0_38,nu_0_39,nu_0_40,calc_effective_T
2912930,0.94,0.016037,0.28,2.1,13.521936,2.467034,3.149089,0.08889,176.128023,213.895231,...,1114.570211,1149.135658,1183.784224,1218.35837,1252.882042,1287.228693,1321.365888,1355.228485,1388.868783,4895.366659


In [12]:
inputs = ['initial_mass', 'initial_Zinit', 'initial_Yinit', 'initial_MLT', 'star_age']

n_min=15
n_max=25
outputs = ['calc_effective_T', 'luminosity', 'star_feh'] + [f'nu_0_{i+1}' for i in range(n_min, n_max+1)]
hare_cut = hare_df[inputs+outputs]
hare_cut

,initial_mass,initial_Zinit,initial_Yinit,initial_MLT,star_age,calc_effective_T,luminosity,star_feh,nu_0_16,nu_0_17,nu_0_18,nu_0_19,nu_0_20,nu_0_21,nu_0_22,nu_0_23,nu_0_24,nu_0_25,nu_0_26
2912930,0.94,0.016037,0.28,2.1,13.521936,4895.366659,3.149089,0.08889,584.987796,619.270851,653.65927,688.124936,722.611306,757.088263,791.415761,824.436728,857.505374,889.175268,919.03608


In [9]:
teff_unc = 70 #K
luminosity_unc = 0.04 #L\odot
surface_feh_unc = 0.1 #dex
frequency_unc = 0.5 #\muHz

obs_unc = np.array([teff_unc, luminosity_unc, surface_feh_unc]+[frequency_unc for i in range(n_min,n_max+1)])

In [17]:
def obs_noise(true, unc, seed=None):
    seeded_random_state = np.random.RandomState(seed=seed)
    rvs_random_states = seeded_random_state.randint(0, high=2**32 -1,size=len(true))
    noisy_obs = np.empty(len(true))
    idx = 0
    for ob in true:
        noisy_obs[idx] = scipy.stats.norm(loc=ob, scale=unc[idx]).rvs(random_state=rvs_random_states[idx])
        idx+=1

    return noisy_obs

hare_obs = obs_noise(hare_cut[outputs].values[0],obs_unc)
hare_obs

array([ 4.71558596e+03,  1.28710936e+01, -3.72038214e-01,  1.93979470e+02,
        2.05847119e+02,  2.15130022e+02,  2.26618919e+02,  2.35091823e+02,
        2.44214187e+02,  2.55911811e+02,  2.66620211e+02,  2.78238385e+02,
        2.91065212e+02,  3.02424813e+02])

## all hares in form

In [50]:
import os
inputs = ['initial_mass', 'initial_Zinit', 'initial_Yinit', 'initial_MLT', 'star_age']

n_min=15
n_max=25
outputs = ['calc_effective_T', 'luminosity', 'star_feh'] + [f'nu_0_{i+1}' for i in range(n_min, n_max+1)]

teff_unc = 70 #K
luminosity_unc = 0.04 #L\odot
surface_feh_unc = 0.1 #dex
frequency_unc = 0.5 #\muHz

obs_unc = np.array([teff_unc, luminosity_unc, surface_feh_unc]+[frequency_unc for i in range(n_min,n_max+1)])

def obs_noise(true, unc, seed=None):
    seeded_random_state = np.random.RandomState(seed=seed)
    rvs_random_states = seeded_random_state.randint(0, high=2**32 -1,size=len(true))
    noisy_obs = np.empty(len(true))
    idx = 0
    for ob in true:
        noisy_obs[idx] = scipy.stats.norm(loc=ob, scale=unc[idx]).rvs(random_state=rvs_random_states[idx])
        idx+=1

    return noisy_obs

for hare_idx in range(100):
    hare_df = pd.read_json(f'form/hare{hare_idx}/hare{hare_idx}.json')
    hare_cut = hare_df[inputs+outputs]
    hare_obs = obs_noise(hare_cut[outputs].values[0],obs_unc)
    hare_obs_df = hare_cut.copy()
    hare_obs_df[outputs] = hare_obs

    path = f'form/hare{hare_idx}/obs0'
    if not os.path.exists(path):
      os.mkdir(path)
      print(f"{path} created!")
    else:
      print(f"{path} already exists", end='\r')
    hare_obs_df.to_json(path+f'/obs0.json')

form/hare1/hare1-obs0 created!exists
form/hare2/hare2-obs0 created!
form/hare3/hare3-obs0 created!
form/hare4/hare4-obs0 created!
form/hare5/hare5-obs0 created!
form/hare6/hare6-obs0 created!
form/hare7/hare7-obs0 created!
form/hare8/hare8-obs0 created!
form/hare9/hare9-obs0 created!
form/hare10/hare10-obs0 created!
form/hare11/hare11-obs0 created!
form/hare12/hare12-obs0 created!
form/hare13/hare13-obs0 created!
form/hare14/hare14-obs0 created!
form/hare15/hare15-obs0 created!
form/hare16/hare16-obs0 created!
form/hare17/hare17-obs0 created!
form/hare18/hare18-obs0 created!
form/hare19/hare19-obs0 created!
form/hare20/hare20-obs0 created!
form/hare21/hare21-obs0 created!
form/hare22/hare22-obs0 created!
form/hare23/hare23-obs0 created!
form/hare24/hare24-obs0 created!
form/hare25/hare25-obs0 created!
form/hare26/hare26-obs0 created!
form/hare27/hare27-obs0 created!
form/hare28/hare28-obs0 created!
form/hare29/hare29-obs0 created!
form/hare30/hare30-obs0 created!
form/hare31/hare31-obs